In [15]:
import torch
import os
import numpy as np  
import faiss  
from deslib.util.faiss_knn_wrapper import FaissKNNClassifier

In [5]:
inputpath = './data/tgnet_resize_dataset/test/MNIAB8K3_lower.pth'

In [13]:
data = torch.load(inputpath)
coord = data['coord']
labels = data['labels']

In [ ]:
# 确定需要取出的元素个数  
num_elements = len(coord) // 5
  
# 随机选择索引  
selected_indices = np.random.choice(len(coord), num_elements, replace=False)  
  
# 假设 points 包含已知点的坐标，labels 包含对应的标签  
# points 的形状为 (n_points, 3)，labels 的形状为 (n_points,)  
# new_points 包含要预测标签的新点的坐标  
  


# 调整数据类型  
points = coord[selected_indices]
points = points.astype('float32')  

mask = np.ones(labels.shape, dtype=bool)
mask[selected_indices] = False  

new_points = coord[mask]
new_points = new_points.astype('float32')  
  


In [19]:
clf = FaissKNNClassifier(n_neighbors=5, n_jobs=10, algorithm='brute', n_cells=100, n_probes=2)
clf.fit(points, labels[selected_indices])  #训练
y_test_proba = clf.predict_proba(new_points) #预测概率
print(y_test_proba)

ValueError: could not broadcast input array from shape (16,) into shape (14,)

In [ ]:
# 建立索引  
index = faiss.GpuIndexFlatL2(3)  # 使用 L2 距离的 Flat 索引  
index.add(points)  # 将已知点添加到索引中  
  
# 搜索最近邻  
k = 5  # 选择邻居的数量  
D, I = index.search(new_points, k)  # D 包含距禽最近邻的距离，I 包含最近邻的索引  
  
# 根据最近邻的标签进行预测  
predicted_labels = labels[I]  
  
print("预测的标签:", predicted_labels)  
print(np.count_nonzero(predicted_labels==labels[mask])/labels[mask].shape[0])